In [60]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import os, sys
sys.path.append('../')
import ama

In [84]:
from keras.optimizers import Adam

In [71]:
Vgg = ama.vgg.Vgg
TrainBatch = ama.trainbatch.TrainBatch

In [63]:
path= '../data/'
cv_version = 1
cv_version = str(cv_version)
batch_size = 128
img_size = (64,64)

In [74]:
vgg = Vgg(input_shape=(3,)+img_size, dropout=0)
vgg.load_weights('../data/weights/atc_1.hk', loaded_dropout=0.25)

In [75]:
valgen = TrainBatch(path+'val-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size)

Found 6000 images belonging to 1 classes.


In [76]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=5,
        horizontal_flip=True, vertical_flip=True)

traingen = TrainBatch(path+'train-jpg'+cv_version+'/', path+'train_v2.csv', batch_size=batch_size, img_size=img_size,
                      imagegen=gen)

Found 34479 images belonging to 1 classes.


In [77]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from ama.persistenthistory import PersistentHistory

earlystop = EarlyStopping(patience=4)
history = PersistentHistory()

In [81]:
def get_saver(fp):
    return ModelCheckpoint(filepath, verbose=1, save_best_only=True, save_weights_only=True)

In [82]:
filepath = '../data/weights/atc'+cv_version+'_less_dropout_best_lr1.hk'
saver = get_saver(filepath)

In [ ]:
vgg.model.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy', metrics=['accuracy'])
vgg.model.fit_generator(traingen, samples_per_epoch=traingen.nb_sample, nb_epoch=20,
                        validation_data=valgen, nb_val_samples=valgen.nb_sample,
                        callbacks=[history, saver, earlystop])

Epoch 1/20
34432/34479 [============================>.] - ETA: 0s - loss: 0.0963 - acc: 0.9622